In [1]:
#Written by GC -- 03/24

#This notebook contains code to generate SEDs dists for each NTS sheet. The basic idea here is to 1) determine the number of days where the FWI exceeds some threshold 
#(see https://doi.org/10.1016/j.scitotenv.2023.161831), 2) calculate the average fire duration per NTS sheet and then multiply these values to get the avg SEDs per NTS sheet. 
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import contextily as cx
import libpysal

In [2]:
#Calc number of days above some FWI value using the ecozone FWI cutoffs found in https://doi.org/10.1016/j.scitotenv.2023.161831

#Read in NTS sheet shapefiles
df_nts=gpd.read_file("C:\\Users\\GiovanniCorti\\Downloads\\nts_snrc\\nts_snrc_250k.shp")

#Read in Ecozne shapefiles. The boreal shield ecozone conatins an EW split unique to this paper, hence the
#extra shape files
df_ecozones=gpd.read_file("C:\\Users\\GiovanniCorti\\Downloads\\ecozone_shp\\Ecozones\\ecozones.shp")
df_bsw=gpd.read_file("C:\\Users\\GiovanniCorti\\Documents\\BSW.shp")
df_bse=gpd.read_file("C:\\Users\\GiovanniCorti\\Documents\\BSE.shp")
df_ecozones=pd.concat([df_ecozones,df_bsw,df_bse],ignore_index=True)
df_ecozones=df_ecozones.drop([15])


#Reproject to equal-area EASE grid
df_nts=df_nts.to_crs(epsg=6931)
df_ecozones=df_ecozones.to_crs(epsg=6931)

C:\Users\GiovanniCorti\AppData\Local\Temp\ipykernel_1500\370250213.py:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_ecozones=pd.concat([df_ecozones,df_bsw,df_bse],ignore_index=True)


In [ ]:
#Add FWI cutoff values to ecozone dataframe. Where https://doi.org/10.1016/j.scitotenv.2023.161831 
#does not provide an FWI value I just assume 19
tdf=df_ecozones.copy()
FWI_50=[19,19,19,19,19,19,10.5,17.7,19.6,16.7,20.8,19,11.5,11.5,9.5,11.5,23.2,30,11.5,19,20.8,19,19,19,15.8,12]
tdf['FWI_50']=FWI_50
FWI50_dict=pd.Series(tdf['FWI_50'].values,index=tdf['ZONE_NAME']).to_dict()

#Checks NTS sheets to see if they stradle ecozone boundaries and takes weighted average for more accurate 
#FWI cutoff values if needed
nts_list,FWI_cf_list=[],[]
for index, row in df_nts.iterrows():
    int_poly=df_ecozones.intersection(row.geometry).area
    ar=int_poly/np.sum(int_poly)
    tdf['Area_ratio']=ar
    ar_dict=tdf.groupby('ZONE_NAME')['Area_ratio'].sum().to_dict()
    FWI_cf=sum(ar_dict[k]*FWI50_dict[k] for k in ar_dict)
    FWI_cf_list.append(FWI_cf)
    nts_list.append(row['NTS_SNRC'])

#Create FWI cutoff dataframe
dict = {'NTS_SNRC': nts_list, 'FWI_cf': FWI_cf_list} 
df_fwi_cf=pd.DataFrame(dict)
df_nts_fwi_cf=df_nts.merge(df_fwi_cf, on='NTS_SNRC')

df_nts_fwi_cf=df_nts_fwi_cf.to_crs(epsg=3857)

In [ ]:
#Threshold FWI files and save to Y: drive
for index, row in df_nts_fwi_cf.iterrows():
    NTS_code=row['NTS_SNRC']
    FWI_cf=row['FWI_cf']
    dir_path="Y:\\client-data\\demo_projects\\climate85\\Working_data\\NARR_weather_csvs\\NTS_SNRC_"+NTS_code
    if os.path.exists(dir_path):
        #Read in fwi_era data
        fwi_fp=dir_path+"\\fwi_era_NTS_SNRC_"+NTS_code+".csv"
        fwi_df=pd.read_csv(fwi_fp)
        #Threshold data and save to Y: drive
        fwi_cf_df=fwi_df[fwi_df['fwi']>FWI_cf]
        fwi_cf_df.to_csv(dir_path+"\\fwi_era_cf_NTS_SNRC_"+NTS_code+".csv")


In [11]:
#Read in number of node-days (ERA5 nodes per NTS sheet). Used to calc percentage of ERA5 days above some FWI cutoff
nd_df=pd.read_csv(r"C:\\Users\\GiovanniCorti\\Downloads\\node_days.csv")
df_nts_fwi_cf=df_nts_fwi_cf.merge(nd_df, on='NTS_SNRC')
#214 is number of days between 1 April and 1 Nov, an interval that contains >99% of the fires in the NFDB
df_nts_fwi_cf['met_samples']=df_nts_fwi_cf['node_days']*214*11

#Include only NTS sheets where an ign is defined
df_igns=gpd.read_file(r"C:\Users\GiovanniCorti\Documents\Wildfire\ign_v2.shp")
df_nts=df_igns[df_igns['ign_num']>0]

#For each NTS sheet, read ERA5 weather CSVs and calc percentage above FWI cutoff
nts_list,per_list=[],[]
for index, row in df_nts_fwi_cf.iterrows():
    nts_code=row['NTS_SNRC']
    nts_list.append(nts_code)
    FWI_df=pd.read_csv("Y:\\client-data\\demo_projects\\climate85\\Working_data\\NARR_weather_csvs\\NTS_SNRC_"+nts_code+"\\fwi_era_NTS_SNRC_"+nts_code+".csv")
    per_list.append(len(FWI_df[FWI_df['fwi']>row['FWI_cf']])/row['met_samples'])

#Create geodataframe w/ percentage FWI above
dict = {'NTS_SNRC': nts_list, 'fwi_per': per_list} 
tdf=pd.DataFrame(dict)
FWI_above_df=df_nts.merge(tdf, on='NTS_SNRC')

In [194]:
#Now that we have percenatge of FWI above cutoff we need average fire duration. This is done on an ecozone basis using fire progression data

nfdb_df=gpd.read_file("C:\\Users\\GiovanniCorti\\Documents\\Wildfire\\AreaBurned2010-2020\\NBAC_2010_2020.shp")
nfdb_df['EDATE']=pd.to_datetime(nfdb_df['EDATE'],exact=False)
nfdb_df['SDATE']=pd.to_datetime(nfdb_df['SDATE'],exact=False)

nfdb_df['Duration']=nfdb_df['EDATE']-nfdb_df['SDATE']

#Replace 0 day duration with 1 day duration
nfdb_df['Duration']=nfdb_df['Duration'].where(nfdb_df['Duration']!=np.timedelta64(0, 'D'),np.timedelta64(1, 'D'))
#Assume smalls fires (less than 10 ha) with no duration have 1 day duration
nfdb_df.loc[(nfdb_df['POLY_HA'] <10) & (np.isnat(nfdb_df['Duration'])),'Duration']=np.timedelta64(1, 'D')

In [ ]:
#Read in ecozone shapefile and reproject to matching CRS. These are the standard ecozones instead 
#of the split boreal-sheild version used above
ecozone_df=gpd.read_file("C:\\Users\\GiovanniCorti\\Downloads\\ecozone_shp\\Ecozones\\ecozones.shp")
ecozone_df=ecozone_df.to_crs(nfdb_df.crs)

#Determine which ecozone each fire is in
#Size cutoff here is that same as used in the ign dist calculation and is 
#inteded, in part, to implicitly account for fire supression 
nfdb_lg_df=nfdb_df[nfdb_df['POLY_HA']>1]
for index, row in nfdb_lg_df.iterrows():
    a=ecozone_df.intersection(row['geometry'].centroid)
    ez_num=ecozone_df.iloc[a[~a.is_empty].index[0]]['ECOZONE']
    nfdb_lg_df.loc[index,'ECOZONE']=ez_num

#Groupby ecozone and calc average duration
#Round long fires down to 30 days
nfdb_lg_df.loc[nfdb_lg_df['Duration'] > np.timedelta64(30, 'D') ,'Duration']=np.timedelta64(30, 'D')
dur_df=nfdb_lg_df.groupby('ECOZONE')['Duration'].mean()

In [210]:
#df_nts=gpd.read_file("C:\\Users\\GiovanniCorti\\Downloads\\nts_snrc\\nts_snrc_250k.shp")
#ecozone_df=gpd.read_file("C:\\Users\\GiovanniCorti\\Downloads\\ecozone_shp\\Ecozones\\ecozones.shp")

#Reproj to equal-area CRS before we do calculation for sheets that straddle ecozone boundaries
df_nts=df_nts.to_crs(epsg=6931)
ecozone_df=ecozone_df.to_crs(epsg=6931)

#Merge duration dataframe with ecozone dataframe
tdf=ecozone_df.copy()
tdf=tdf.merge(dur_df, on='ECOZONE')

#Setup dict w/ avg duration for ecozone
dur_dict=pd.Series(tdf['Duration'].values,index=tdf['ZONE_NAME']).to_dict()

#Calc fire duration for each NTS sheet in a way that accounts for NTS sheets that straddle 
#ecozone boundaries
nts_list,fd_list=[],[]
for index, row in df_nts.iterrows():
    int_poly=ecozone_df.intersection(row.geometry).area
    ar=int_poly/np.sum(int_poly)
    tdf['Area_ratio']=ar
    ar_dict=tdf.groupby('ZONE_NAME')['Area_ratio'].sum().to_dict()
    fd=sum(ar_dict[k]*(dur_dict[k]/np.timedelta64(1, 'D')) for k in ar_dict)
    fd_list.append(fd)
    nts_list.append(row['NTS_SNRC'])

#Use ign dataframe here so we can skip NTS sheets w/ no fires.
df_igns=gpd.read_file(r"C:\Users\GiovanniCorti\Documents\Wildfire\ign_v2.shp")
df_nts=df_igns[df_igns['ign_num']>0]
dict = {'NTS_SNRC': nts_list, 'avg_fd': fd_list} 
df_fd=pd.DataFrame(dict)
df_fd=df_nts.merge(df_fd, on='NTS_SNRC')

In [211]:
#Round min fire duration to 1 and calc avg SEDs
df_fd.loc[df_fd['avg_fd'].between(0.01,1),'avg_fd']=1
df_SED=df_fd[['NTS_SNRC','avg_fd']].merge(FWI_above_df, on='NTS_SNRC')

#-1 here is used as I calc SEDs in excess of 1 and then add the inital SED back in. 
#This generally ensures a min of at least 1 SED.
df_SED['SED']=(df_SED['fwi_per']*(df_SED['avg_fd']-1))+1

#Spatial smoothing using nearest 8 NTS sheets
W = libpysal.weights.KNN.from_dataframe(df_SED, k=8)
# row-normalise weights
W.transform = "r"
df_SED["SED_sm"] = libpysal.weights.lag_spatial(W, df_SED["SED"])

In [212]:
df_SED = gpd.GeoDataFrame(df_SED, crs="EPSG:6931", geometry='geometry')
df_SED.loc[df_SED['SED_sm'].le(1),'SED_sm']=1
df_SED.to_file("C:/Users/GiovanniCorti/Documents/Wildfire/SED_v2.shp")

In [217]:
#Create SED dist .csvs for each NTS sheet. These csvs are not possion distributed and only feature 2 possible SED values.
for index, row in df_SED.iterrows():
    num=np.round(row['SED_sm'],2)
    vals=np.round(np.modf(num),2)
    SED_num_ls=[]
    per_ls=[]
    
    if num==0:
        #r1=[0,1.0]
        SED_num_ls.append(0)
        per_ls.append(100)
    elif np.isnan(num):
        pass
    else:
        SED_num_ls.extend((int(vals[1]),int(vals[1]+1)))
        per_ls.extend((100*np.round(1-vals[0],2),100*np.round(vals[0],2)))
    dict = {'sp_ev_days': SED_num_ls, 'pct': per_ls} 
    dt=pd.DataFrame(dict)
    #Write .csvs to Y drive
    dt.to_csv("Y:client-data\\demo_projects\\climate85\\Working_data\\SED_dist_v2\\sed_dist_"+row['NTS_SNRC']+'.csv',index=False)